# Preprocessing

In [9]:
import sys
import codecs
import json
import re
import operator 
import requests
from collections import Counter
import nltk

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from nltk import bigrams

emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via'] 
fname = 'C:\\Users\\Chirantan\\Desktop\\tweets\\python.json'

with open(fname, 'r') as f:
    count_all = Counter()
    for line in f:
        try:
            tweet = json.loads(line)
        except:
            continue
        # Create a list with all the terms
        try:
            terms_stop = [term for term in preprocess(tweet['text']) if term not in stop]
        except:
            continue

        # Update the counter
	    #terms_single = set(terms_all)
		# Count hashtags only
        try:
            terms_hash = [term for term in preprocess(tweet['text']) 
              if term.startswith('#')]
        except:
            continue
# Count terms only (no hashtags, no mentions)

        try:
            terms_only = [term for term in preprocess(tweet['text']) 
              if term not in stop and
              not term.startswith(('#', '@'))] 
        except:
            continue
              
              # startswith() takes a tuple (not a list) if 
              # we pass a list of inputs
        terms_single=set(terms_stop)
        terms_bigram = bigrams(terms_stop)	
        count_all.update(terms_stop)
        
    # Print the first few most frequent words
    print(count_all.most_common(20))



[('RT', 9355), ('…', 9006), ('Hollywood', 5953), ('The', 3121), ('hollywood', 2519), ('shit', 2215), ('LA', 2209), ('always', 2188), ('everyone', 2154), ('los', 2147), ('bar', 2135), ('REAL', 2127), ('none', 2124), ('vegan', 2123), ('fruit', 2121), ('angeles', 2120), ('@RaulsBrainLeft', 2120), ('https://t.co/CPdE', 2119), ('labeling', 2119), ('’', 1660)]


# Sentiment Analysis

In [8]:
import json
import re
import operator 
from textblob import TextBlob
from collections import Counter
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
import os, sys, codecs
from nltk import bigrams
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via'] 
fname = 'C:\\Users\\Chirantan\\Desktop\\tweets\\python.json'
with open(fname, 'r') as f:
    lis=[]
    neg=0.0
    n=0.0
    net=0.0
    pos=0.0
    p=0.0
    count_all = Counter()
    cout=0
    for line in f:
        try:
            tweet = json.loads(line)
        except:
            continue
        # Create a list with all the terms
        try:
            blob = TextBlob(tweet["text"])
            cout+=1
            lis.append(blob.sentiment.polarity)
        except:
            continue
        #print blob.sentiment.subjectivity
        #print (os.listdir(tweet["text"]))
        if blob.sentiment.polarity < 0:
            sentiment = "negative"
            neg+=blob.sentiment.polarity
            n+=1
        elif blob.sentiment.polarity == 0:
            sentiment = "neutral"
            net+=1
        else:
            sentiment = "positive"
            pos+=blob.sentiment.polarity
            p+=1

        # output sentiment
        
    print ("Total tweets",len(lis))
    print ("Positive ",float(p/cout)*100,"%")
    print ("Negative ",float(n/cout)*100,"%")
    print ("Neutral ",float(net/len(lis))*100,"%")
    #print lis
        # determine if sentiment is positive, negative, or neutral
        
        # output sentiment
        #print sentiment

Total tweets 15002
Positive  31.58245567257699 %
Negative  14.184775363284896 %
Neutral  54.23276896413811 %
